In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import re
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
# Get the current working directory
path = 'drive/My Drive/Colab Notebooks'

print(path)

# Open files
filename = path+'/Udiv-Lelegans.anchors.no-interanchors.no-local.txt'
with open(filename) as f:
    reader = csv.reader(f, delimiter="\t")
    Udiv_Lelegans_data = list(reader)

filename = path+'/Udiv-Dplan.anchors.unfiltered.txt'
with open(filename) as f:
    reader = csv.reader(f, delimiter="\t")
    Udiv_Dplan_data = list(reader)

filename = path+'/Udiv-Mbourn.anchors.unfiltered.txt'
with open(filename) as f:
    reader = csv.reader(f, delimiter="\t")
    Udiv_Mbourn_data = list(reader)

filename = path+'/sex_genes.txt'
with open(filename) as f:
    reader = csv.reader(f, delimiter="\t")
    sex_list = list(reader)



drive/My Drive/Colab Notebooks


In [ ]:
# Sex scaffold IDs

Lelegans_sex = 'chr_1','chr_9'
Dplan_sex = 'chrom-5','chrom-12'
Mbourn_sex = 'X1', 'X2'



In [ ]:
def sex_search(gene_list, sex_list):

  genes_found = []
  for count, gene in enumerate(gene_list):

    r = count % 100
    if r==0:
      print(count)
    for line in sex_list:
      if bool(re.search(line[0], gene)):
        genes_found.append(gene)
        break
        

  return genes_found


In [ ]:
def search_scaffolds(data,sex_scaffolds,sex_list):

  sex_list = []
  synteny_list = []

  line = 0
  while (line < len(data)):
    begin = 0
    last = 0
    
    if bool(re.fullmatch('scaffold_3', data[line][0])) or bool(re.fullmatch('scaffold_10', data[line][0])):

      if bool(re.search(sex_scaffolds[0], data[line][3])) or bool(re.search(sex_scaffolds[1], data[line][3])):
          
        synteny_list.append(data[line][7])
        sex_gene = sex_search(data[line][7], sex_list)

        if sex_gene:
          sex_list.append(sex_gene)
    
    line += 1

  return sex_list, synteny_list



In [ ]:
def make_gene_list(gene_start, gene_end):

  genes = list(np.arange(gene_start,gene_end+1))

  for count, x in enumerate(genes):
    genes[count] = 'mRNA'+str(genes[count])

  return genes

In [ ]:
def search_scaffolds2(data,sex_scaffolds):

  synteny_list = []

  line = 0
  while (line < len(data)):

    if bool(re.fullmatch('#block begin', data[line][0])):
      if bool(re.fullmatch('scaffold_3', data[line+1][0])) or bool(re.fullmatch('scaffold_10', data[line+1][0])):
        if bool(re.search(sex_scaffolds[0], data[line+1][3])) or bool(re.search(sex_scaffolds[1], data[line+1][3])):

          gene_start = data[line+1][7]
          gene_start = int(gene_start[4:])
    
    if bool(re.fullmatch('#block end', data[line][0])):
      if bool(re.fullmatch('scaffold_3', data[line-1][0])) or bool(re.fullmatch('scaffold_10', data[line-1][0])):
        if bool(re.search(sex_scaffolds[0], data[line-1][3])) or bool(re.search(sex_scaffolds[1], data[line-1][3])):

          gene_end = data[line-1][7]
          gene_end = int(gene_end[4:])

          gene_list = make_gene_list(gene_start, gene_end)

          if synteny_list:
            synteny_list = synteny_list + gene_list
          else:
            synteny_list = gene_list
    
    line += 1

  return synteny_list



In [ ]:
Lelegans_synteny_list = search_scaffolds2(Udiv_Lelegans_data,Lelegans_sex)
Dplan_synteny_list = search_scaffolds2(Udiv_Dplan_data,Dplan_sex)
Mbourn_synteny_list = search_scaffolds2(Udiv_Mbourn_data,Mbourn_sex)


Lelegans_synteny_list = list(set(Lelegans_synteny_list)) 
Dplan_synteny_list = list(set(Dplan_synteny_list)) 
Mbourn_synteny_list = list(set(Mbourn_synteny_list)) 

#Lelegans_sex_list = sex_search(Lelegans_synteny_list[0:100], sex_list)
#Dplan_sex_list = sex_search(Dplan_synteny_list, sex_list)
#Lelegans_sex_list = sex_search(Mbourn_synteny_list[0:100], sex_list)


In [ ]:

Dplan_new_list = list(set(Dplan_synteny_list)) 
Lelegans_new_list = list(set(Lelegans_synteny_list)) 
Mbourn_new_list = list(set(Mbourn_synteny_list)) 

print(len(Dplan_synteny_list))
print(len(Dplan_new_list))
print(len(Lelegans_synteny_list))
print(len(Lelegans_new_list))
print(len(Mbourn_synteny_list))
print(len(Mbourn_new_list))



5818
5818
6367
6367
5403
5403
